In [1]:
import pandas as pd 
import numpy as np 
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

# Precisamos fazer isso para importar variávies de outro notebook.
%store -r credits
%store -r movies

C:\Users\renan\AppData\Roaming\Python\Python312\site-packages\IPython\extensions\storemagic.py:148: UserWarning: This is now an optional IPython functionality, using autorestore/credits requires you to install the `pickleshare` library.
  obj = db["autorestore/" + arg]
C:\Users\renan\AppData\Roaming\Python\Python312\site-packages\IPython\extensions\storemagic.py:148: UserWarning: This is now an optional IPython functionality, using autorestore/movies requires you to install the `pickleshare` library.
  obj = db["autorestore/" + arg]


In [2]:
df1=pd.read_csv(r"tmdb_5000_credits.csv",encoding='utf-8', sep=',')
df2=pd.read_csv(r"tmdb_5000_movies.csv",encoding='utf-8', sep=',')

df1.columns = ['id','title','cast','crew']
dados = df2.merge(df1,on='id')

dados.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


# Content Based Filtering

Neste sistema de recomendação, o conteúdo do filme (sinopse, elenco, equipe, palavra-chave, slogan etc.) é usado para encontrar sua semelhança com outros filmes. Em seguida, os filmes que são mais provavelmente semelhantes são recomendados.

In [3]:
dados['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [13]:
# Inicializando o objeto Vetorizador TF-IDF com a remoção de stopwords em inglês
vetorizador_tfidf = TfidfVectorizer(stop_words='english')

# Preenchendo valores nulos na coluna 'visão geral' com uma string vazia
dados['overview'] = dados['overview'].fillna('')

# Criando uma matriz TF-IDF a partir dos textos da coluna 'visão_geral'
# matriz_tfidf = vetorizador_tfidf.fit_transform(dados['overview'])

vetorizador_tfidf = TfidfVectorizer(stop_words='english')

# Vetorizando a coluna 'overview'
overview_tfidf = vetorizador_tfidf.fit_transform(dados['overview']).toarray()

# Vetorizando a coluna 'title'
title_tfidf = vetorizador_tfidf.fit_transform(dados['title_x']).toarray()

# Combinando as matrizes TF-IDF
combined_tfidf = np.hstack((title_tfidf, overview_tfidf))




# Extraindo a forma (shape) da matriz TF-IDF
# Essa linha retorna uma tupla representando o número de linhas (documentos) e colunas (features) na matriz TF-IDF
print("Forma da Matriz TF-IDF:", combined_tfidf.shape)

Forma da Matriz TF-IDF: (4803, 25472)


In [14]:
# Calculando a similaridade cosseno entre as linhas da matriz TF-IDF
similaridade_cosseno = linear_kernel(combined_tfidf, combined_tfidf)
print(similaridade_cosseno)

# Criando uma série de índices onde os índices são os títulos dos documentos e os valores são os índices correspondentes no DataFrame
indices = pd.Series(dados.index, index=dados['title_x']).drop_duplicates()

[[2.         0.         0.         ... 0.         0.         0.        ]
 [0.         2.         0.         ... 0.02160533 0.         0.        ]
 [0.         0.         2.         ... 0.01488159 0.         0.        ]
 ...
 [0.         0.02160533 0.01488159 ... 2.         0.01609091 0.00701914]
 [0.         0.         0.         ... 0.01609091 2.         0.01171696]
 [0.         0.         0.         ... 0.00701914 0.01171696 2.        ]]


In [40]:
def obter_recomendacoes(titulo, similaridade_cosseno=similaridade_cosseno):
    if titulo not in indices:
        return "Filme não encontrado."
    indice = indices[titulo]
    scores_similares = list(enumerate(similaridade_cosseno[indice]))
    scores_similares = sorted(scores_similares, key=lambda x: x[1], reverse=True)

    # Faz com que mostrem filmes que tenham um score de similaridade acima de 0.6
    scores_similares = [score for score in scores_similares[1:] if score[1] > 0.6]
    indices_filmes_recomendados = [i[0] for i in scores_similares]

      # Se nenhum filme atender ao critério, você pode optar por retornar uma mensagem ou lista vazia
    if not indices_filmes_recomendados:
        return "Nenhum filme similar encontrado."
    
    # Certifique-se de que você está acessando os nomes de colunas corretos
    recomendacoes = dados.loc[indices_filmes_recomendados, ['title_x', 'overview']]
    recomendacoes['cosine_similarity'] = [s[1] for s in scores_similares]  # adicionando valores de similaridade cosseno
    
    return recomendacoes


In [41]:
obter_recomendacoes('The Fast and the Furious')

,title_x,overview,cosine_similarity
500,2 Fast 2 Furious,It's a major double-cross when former police o...,1.086473
204,Fast Five,Former cop Brian O'Conner partners with ex-con...,1.042821
44,Furious 7,Deckard Shaw seeks revenge against Dominic Tor...,0.954234
405,The Fast and the Furious: Tokyo Drift,"In order to avoid a jail sentence, Sean Boswel...",0.680311


In [42]:
obter_recomendacoes('Titanic')

,title_x,overview,cosine_similarity
1269,Raise the Titanic,"To obtain a supply of a rare mineral, a ship r...",0.836647
3103,The Chambermaid on the Titanic,"Horty, a French foundry worker, wins a contest...",0.748169
